In [1]:
import pandas as pd
import requests
from keras.layers import Dense, Bidirectional, LSTM, Input, Embedding
from keras.models import Model

Using TensorFlow backend.


In [83]:
# words
# characters
# phonemes
# document id
# author id
CHAR_LEN = 10
PHONEME_LEN = 4

words = Input()
chars = Input()
phonemes = Input()
doc = Input()
auth = Input()

word_embed = Embedding()(words)
char_lstm = Bidirectional(LSTM(CHAR_LEN, return_sequences=False), input_shape=(5, CHAR_LEN))(chars)
phoneme_lstm = Bidirectional(LSTM(PHONEME_LEN, return_sequences=False), input_shape=(5, PHONEME_LEN))(phonemes)





x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training

ConnectionError: HTTPSConnectionPool(host='poetrydb.org', port=443): Max retries exceeded with url: /authors (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0xb3bc56890>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [85]:
import nltk

ImportError: No module named nltk

In [2]:
r = requests.get("http://poetrydb.org/author")

ConnectionError: HTTPConnectionPool(host='poetrydb.org', port=80): Max retries exceeded with url: /author (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0xb37175210>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [ ]:
1